In [71]:
import os
from datetime import datetime
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('future.no_silent_downcasting', True)
import seaborn as sns
import matplotlib.pyplot as plt

In [72]:
data = pd.read_csv("data/marketing_campaign.csv", sep="\t")
print("Number of datapoints:", len(data))
data.head()

Number of datapoints: 2240


,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


In [73]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

Income has Null values - Delete these coloumns

In [74]:
print("Total categories in the feature Marital_Status:\n", data["Marital_Status"].value_counts(), "\n")
print("Total categories in the feature Education:\n", data["Education"].value_counts())

Total categories in the feature Marital_Status:
 Marital_Status
Married     864
Together    580
Single      480
Divorced    232
Widow        77
Alone         3
Absurd        2
YOLO          2
Name: count, dtype: int64 

Total categories in the feature Education:
 Education
Graduation    1127
PhD            486
Master         370
2n Cycle       203
Basic           54
Name: count, dtype: int64


**Data Cleaning**

In [75]:
# Check for duplicates
duplicates = data.duplicated()
# Show the duplicated rows
print(len(data[duplicates]))

0


In [76]:
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)].index
    return list(outliers)

In [77]:
#outliersIncome = detect_outliers_iqr(data, 'Income')

In [78]:
#print(outliersIncome)
#values = data.loc[outliersIncome, 'Income']
#print(values)

In [79]:
#data = data.drop(outliersIncome)

In [80]:
data.shape

(2240, 29)

In [81]:
#Drop NAN Values in Income
data = data.dropna()
data = data.dropna(subset=['Income'])

In [82]:
index_to_drop = data['Income'].idxmax()
print (index_to_drop)
# Drop the row with the highest income
data.drop(index_to_drop, inplace=True)

2233


In [83]:
# Drop rows with none sense input in Marital_Status
indices_yolo = data[data['Marital_Status'] == 'YOLO'].index
data = data.drop(indices_yolo)
indices_absurd = data[data['Marital_Status'] == 'Absurd'].index
data = data.drop(indices_absurd)

**Data Preprocessing**

#Coloumns to Preprocess:
*  Martial_Status (Marek)
*  Education (Marek)
*  Year_Birth (last enrollment date) (Felix)


#Encoding Object Coloumns: (Marek)

#Scaling Features (Felix)

#Feature Engineering
* Living Alone/Living With someone - boolean (Felix)
* Total Amount Spent (Marek)



In [84]:
def get_unique(dataframe):
  for column in df.columns:
    unique_count = df[column].nunique()
    print(f"Number of unique values in column '{column}': {unique_count}")


In [85]:
data['Dt_Customer'] = pd.to_datetime(data['Dt_Customer'], format='%d-%m-%Y')
latest_date = data['Dt_Customer'].max()
print(latest_date)

2014-06-29 00:00:00


In [86]:
#Preprocessing Dt_Customer to numerical format
data['Dt_Customer'] = data['Dt_Customer'].apply(lambda x: x.toordinal())

In [87]:
# Preprocessing Year_Birth
#The Year 2014 is the last included in the dataset -> we choose 2015 as base year
data['Age']=2015-data['Year_Birth']

In [88]:
indices_age = data[data['Age'] > 90].index
print(indices_age)
data = data.drop(indices_age)

Index([192, 239, 339], dtype='int64')


In [89]:
# Create numeric variables for categorical variables and summarize categories.
data["Marital_Categories"] = data["Marital_Status"].replace({"Single":0, "Alone":0, "Together":1, "Married":1, "Divorced":0, "Widow":0})
data["Edu_Categories"] = data["Education"].replace({"Basic":0, "2n Cycle":2, "Graduation":1, "Master":2, "PhD":3})

In [90]:
# Summarize the amount spent on all products.
data["Spent_Total"] = data["MntWines"] + data["MntFruits"] + data["MntMeatProducts"] + data["MntFishProducts"] + data["MntSweetProducts"] + data["MntGoldProds"]

In [91]:
#creating feature "Living_Alone" as boolean
#data['Living_Alone'] = data['Marital_Status'].replace({"Married":1, "Together":1, "Absurd":0, "Widow":0, "YOLO":0, "Divorced":0, "Single":0, "Alone":0})
#data['Living_Alone'] = data['Living_Alone'] + data['Kidhome'] + data['Teenhome']
#data['Living_Alone'] = data['Living_Alone'].where(data['Living_Alone'] <= 1, 1)
#print(data['Living_Alone'].unique())

In [92]:
#creating feature "Having_Kids" as boolean
#data['Having_Kids'] = data['Kidhome'] + data['Teenhome']
#data['Having_Kids'] = data['Having_Kids'].where(data['Having_Kids'] <= 1, 1)

In [93]:
# Summarize the AcceptedOffer
data['AcceptedOffer'] = data['Response'] + data['AcceptedCmp1'] + data['AcceptedCmp2'] + data['AcceptedCmp3'] + data['AcceptedCmp4'] + data['AcceptedCmp5']

In [94]:
print(data['AcceptedOffer'].unique())

[1 0 3 2 4 5]


'Z_CostContact' & Z_Revenue' only have one value, so we can drop the columns.

In [95]:
columns_to_drop = ['Z_CostContact', 'Z_Revenue']
data = data.drop(columns=columns_to_drop)

#Drop redundant features

In [96]:
outliersSpent = detect_outliers_iqr(data, 'Spent_Total')

In [97]:
print(outliersSpent)
values = data.loc[outliersSpent, 'Spent_Total']
print(values)

[1179, 1492, 1572]
1179    2525
1492    2524
1572    2525
Name: Spent_Total, dtype: int64


In [98]:
# Drop the row with the outliers Spent total
#data.drop(outliersSpent, inplace=True)

In [99]:
#Year_Birth and Age are redundant
#Education and Edu_Categories are redundant
#We will use Kids_Home instead of TeenHome and Kidhome
#Living_Alone instead of Marital_Status 
#columns_to_drop = ['Year_Birth', 'Education', 'Kidhome', 'Teenhome', 'Marital_Status']
columns_to_drop = ['Year_Birth', 'Education', 'Marital_Status']
data = data.drop(columns=columns_to_drop)
#drop columns so its not overvalued in the algorithm
#columns_to_drop = ['Response', 'AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5']
#data = data.drop(columns=columns_to_drop)

In [100]:
data.head()

,ID,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,...,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Age,Marital_Categories,Edu_Categories,Spent_Total,AcceptedOffer
0,5524,58138.0,0,0,734750,58,635,88,546,172,...,0,0,0,0,1,58,0,1,1617,1
1,2174,46344.0,1,1,735300,38,11,1,6,2,...,0,0,0,0,0,61,0,1,27,0
2,4141,71613.0,0,0,735101,26,426,49,127,111,...,0,0,0,0,0,50,1,1,776,0
3,6182,26646.0,1,0,735274,26,11,4,20,10,...,0,0,0,0,0,31,1,1,53,0
4,5324,58293.0,1,0,735252,94,173,43,118,46,...,0,0,0,0,0,34,1,3,422,0


In [101]:
def download_dataframe_as_csv(data):
    downloads_folder = os.path.expanduser("~/Downloads")
    # Generate a timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    
    file_name = f"{timestamp}_unscaled_*.csv"
    file_path = os.path.join(downloads_folder, file_name)
    # Save the DataFrame as a CSV file
    data.to_csv(file_path, sep='\t', index=False)
    
    print("DataFrame has been saved as '{timestamp}_feature_unscaled.csv' in the Downloads folder.")

In [102]:
download_dataframe_as_csv(data)

DataFrame has been saved as '{timestamp}_feature_unscaled.csv' in the Downloads folder.
